In [11]:
# Install dependencies (run once per environment)
%pip install -q dspy pandas python-dotenv

# dataset comes from here: https://www.kaggle.com/datasets/nikitpatel/invoice-ner-dataset?resource=download


Note: you may need to restart the kernel to use updated packages.


In [19]:
# Basic imports and environment setup
import os
import json
import dspy
import pandas as pd
from dotenv import load_dotenv

# Load API keys from .env (OPENAI_API_KEY is expected)
load_dotenv()

# Configure DSPy default LM similar to other notebooks
lm = dspy.LM("openai/gpt-5-mini", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000)
dspy.configure(lm=lm)

print("DSPy configured for invoice extraction.")


DSPy configured for invoice extraction.


In [20]:
# Load dataset and preview a few rows
# Prefer absolute path next to this notebook; fallback to project root
nb_dir = os.path.dirname(os.path.abspath("__file__")) if "__file__" in globals() else os.path.abspath("./dspy")
candidates = [
    os.path.join(nb_dir, "invoice_ner_dataset.csv"),
    os.path.abspath("./dspy/invoice_ner_dataset.csv"),
    os.path.abspath("invoice_ner_dataset.csv"),
]
for p in candidates:
    if os.path.exists(p):
        csv_path = p
        break
else:
    raise FileNotFoundError("invoice_ner_dataset.csv not found in expected locations")

raw_df = pd.read_csv(csv_path)
print("Rows:", len(raw_df))
print(raw_df.head(2))

# Parse Final_Output JSON strings into dicts
raw_df["Final_Output"] = raw_df["Final_Output"].apply(lambda s: json.loads(s))

# Build small train/test splits for quick iteration
# Keep it tiny for a simple example; adjust as needed
train_df = raw_df.iloc[:30].copy()
valid_df = raw_df.iloc[30:40].copy()

def to_examples(df):
    examples = []
    for _, row in df.iterrows():
        examples.append(dspy.Example(text=row["Input"], target=row["Final_Output"]).with_inputs("text"))
    return examples

train_examples = to_examples(train_df)
valid_examples = to_examples(valid_df)

print(f"Train examples: {len(train_examples)}, Valid examples: {len(valid_examples)}")


Rows: 67
                                               Input  \
0  Cream and White Simple Minimalist Catering Ser...   
1  Beige Elegant Professional Business Invoice\n\...   

                                        Final_Output  
0  {"TOTAL_AMOUNT": "$1000", "DUE_AMOUNT": "$550"...  
1  {"INVOICE_NUMBER": "#01234", "BILLED_TO": "Est...  
Train examples: 30, Valid examples: 10


In [28]:
# Define a signature for extracting a JSON dict of invoice fields
class InvoiceExtraction(dspy.Signature):
    """
    Extract key-value invoice fields as a JSON dict from free-form invoice text.
    """
    text: str = dspy.InputField(description="Raw invoice text")
    rationale: str = dspy.OutputField(description="Brief reasoning, list detected fields")
    extracted: dict = dspy.OutputField(description="JSON dict with only UPPERCASE dataset keys and string values")

# Base module (simple Predict)
extractor = dspy.Predict(InvoiceExtraction)

print("Signature and extractor ready.")


Signature and extractor ready.


In [29]:
# Define a per-field accuracy metric: correct_fields / total_gold_fields
from dspy.evaluate import Evaluate


def normalize_dict(d: dict) -> dict:
    if d is None:
        return {}
    def norm_key(k: str) -> str:
        # Uppercase keys to align with dataset convention
        return str(k).strip().upper()
    def norm_val(v: str) -> str:
        s = str(v).strip()
        s = s.replace(",", "")
        s = s.replace("$ ", "$")
        s = " ".join(s.split())  # collapse internal whitespace
        return s
    return {norm_key(k): norm_val(v) for k, v in d.items()}


def field_accuracy_metric(example: dspy.Example, pred: dspy.Prediction, trace=None) -> float:
    # count how many fields are correct
    gold = normalize_dict(example.target)
    got = normalize_dict(getattr(pred, "extracted", {}))
    if not gold:
        return 0.0
    correct = 0
    for k, v in gold.items():
        if k in got and got[k] == v:
            correct += 1
    return correct / len(gold)


# Provide the devset at construction per latest API
evaluate = Evaluate(devset=valid_examples, metric=field_accuracy_metric, ordered=True)

# Standard usage: pass the module directly
initial_score = evaluate(extractor)
print("Initial field accuracy on valid:", initial_score)


2025/09/16 12:44:50 INFO dspy.evaluate.evaluate: Average Metric: 3.228846153846154 / 10 (32.3%)


Initial field accuracy on valid: EvaluationResult(score=32.29, results=<list of 10 results>)


In [30]:
# Compare base vs optimized predictions on a sample
sample = valid_examples[0]
print("Sample comparison:")
print("\nINPUT TEXT:")
print("-" * 80)
print(sample.text[:300], "...")
print("-" * 80)

print("\nBASE MODEL PREDICTION:")
base_pred = extractor(text=sample.text)
print(f"Rationale: {base_pred.rationale}")
print("\nExtracted fields:")
for k,v in (base_pred.extracted or {}).items():
    print(f"{k:20s}: {v}")

print("\nGROUND TRUTH:")
print("\nExtracted fields:")
for k,v in sample.target.items():
    print(f"{k:20s}: {v}")


Sample comparison:

INPUT TEXT:
--------------------------------------------------------------------------------
Green Blue Pink Vintage Retro Freelance Invoice

NO. DESCRIPTION RATE QTY TOTAL
01 Graphic design for website 45 20 900.00
02 Graphic design for social content 45 8 360.00
03 Account management fee 280 1 280.00
04 Two days onsite 640 2 1280.00
SUB TOTAL $ 2820.00
TAX $ 225.60
S&H nil
TOTAL $ 3045.60 ...
--------------------------------------------------------------------------------

BASE MODEL PREDICTION:
Rationale: Detected invoice fields parsed from the text:
- Invoice title
- Invoice number ("NO. 001")
- Invoice date ("02.05.2024")
- From (biller) name, company, email, address, code
- To (client) name, company, email, address, code
- Line items (4 entries) with rate, quantity, totals
- Subtotal, Tax, Shipping (S&H), Total
- Bank / account details (Account Name, Bank, Acc)
- Payment instructions / terms ("Please pay within 60 days", cheques payable)
- Notes text

Extracte

In [34]:
# Optimize with MiPROv2 (recommended in tutorial)
from dspy import MIPROv2

mipro = MIPROv2(metric=field_accuracy_metric, auto="medium")


optimized = mipro.compile(
    extractor,
    trainset=train_examples,
    max_bootstrapped_demos=4,
    minibatch=False
)

# Standard usage: evaluate the optimized module directly
opt_score = evaluate(optimized)
print("Optimized field accuracy on valid:", opt_score)


2025/09/16 12:48:06 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 18
minibatch: False
num_fewshot_candidates: 12
num_instruct_candidates: 6
valset size: 24

2025/09/16 12:48:06 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/09/16 12:48:06 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/09/16 12:48:06 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=12 sets of demonstrations...


Bootstrapping set 1/12
Bootstrapping set 2/12
Bootstrapping set 3/12


 67%|██████▋   | 4/6 [02:00<01:00, 30.21s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/12


 67%|██████▋   | 4/6 [00:18<00:09,  4.68s/it]


...

2025/09/16 13:43:18 INFO dspy.evaluate.evaluate: Average Metric: 6.599753059118385 / 24 (27.5%)
2025/09/16 13:43:18 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 27.5 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 7'].
2025/09/16 13:43:18 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [36.35, 27.5, 21.12, 37.97, 15.47, 20.01, 17.37, 0.0, 19.44, 20.34, 28.72, 36.35, 37.97, 15.74, 37.97, 0.0, 40.24, 27.5]
2025/09/16 13:43:18 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 40.24
2025/09/16 13:43:18 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/09/16 13:43:18 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 19 / 18 =====



Average Metric: 2.87 / 24 (12.0%): 100%|██████████| 24/24 [04:02<00:00, 10.09s/it]

2025/09/16 13:47:20 INFO dspy.evaluate.evaluate: Average Metric: 2.8722477857539777 / 24 (12.0%)
2025/09/16 13:47:20 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 11.97 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 11'].
2025/09/16 13:47:20 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [36.35, 27.5, 21.12, 37.97, 15.47, 20.01, 17.37, 0.0, 19.44, 20.34, 28.72, 36.35, 37.97, 15.74, 37.97, 0.0, 40.24, 27.5, 11.97]
2025/09/16 13:47:20 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 40.24
2025/09/16 13:47:20 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/09/16 13:47:20 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 40.24!


2025/09/16 13:48:08 INFO dspy.evaluate.evaluate: Average Metric: 2.9943223443223443 / 10 (29.9%)


Optimized field accuracy on valid: EvaluationResult(score=29.94, results=<list of 10 results>)


In [42]:
# Evaluate the optimized model on validation set
opt_score = evaluate(optimized)
print("Optimized field accuracy on validation:", opt_score)


2025/09/16 13:54:44 INFO dspy.evaluate.evaluate: Average Metric: 2.9943223443223443 / 10 (29.9%)


Optimized field accuracy on validation: EvaluationResult(score=29.94, results=<list of 10 results>)


In [39]:
# Quick demo on a random validation example
sample = valid_examples[0]
print("Sample comparison:")
print("\nINPUT TEXT:")
print("-" * 80)
print(sample.text[:300], "...")
print("-" * 80)

print("\nBASE MODEL PREDICTION:")
base_pred = extractor(text=sample.text)
print(f"Rationale: {base_pred.rationale}")
print("\nExtracted fields:")
for k,v in (base_pred.extracted or {}).items():
    print(f"{k:20s}: {v}")

print("\nOPTIMIZED MODEL PREDICTION:")
opt_pred = optimized(text=sample.text)
print(f"Rationale: {opt_pred.rationale}")
print("\nExtracted fields:")
for k,v in (opt_pred.extracted or {}).items():
    print(f"{k:20s}: {v}")

print("\nGROUND TRUTH:")
print("\nExtracted fields:")
for k,v in sample.target.items():
    print(f"{k:20s}: {v}")


Sample comparison:

INPUT TEXT:
--------------------------------------------------------------------------------
Green Blue Pink Vintage Retro Freelance Invoice

NO. DESCRIPTION RATE QTY TOTAL
01 Graphic design for website 45 20 900.00
02 Graphic design for social content 45 8 360.00
03 Account management fee 280 1 280.00
04 Two days onsite 640 2 1280.00
SUB TOTAL $ 2820.00
TAX $ 225.60
S&H nil
TOTAL $ 3045.60 ...
--------------------------------------------------------------------------------

BASE MODEL PREDICTION:
Rationale: Detected invoice fields parsed from the text:
- Invoice title
- Invoice number ("NO. 001")
- Invoice date ("02.05.2024")
- From (biller) name, company, email, address, code
- To (client) name, company, email, address, code
- Line items (4 entries) with rate, quantity, totals
- Subtotal, Tax, Shipping (S&H), Total
- Bank / account details (Account Name, Bank, Acc)
- Payment instructions / terms ("Please pay within 60 days", cheques payable)
- Notes text

Extracte

In [40]:
# Compare predictions with ground truth
print("\nField Comparison:")
print("-" * 80)
for field in sample.target.keys():
    gt_value = sample.target[field]
    opt_value = opt_pred.extracted.get(field, "MISSING")
    match = "✓" if gt_value == opt_value else "✗"
    print(f"{field:20s}: {match}")
    if match == "✗":
        print(f"  Ground truth: {gt_value}")
        print(f"  Prediction:   {opt_value}")
print("-" * 80)



Field Comparison:
--------------------------------------------------------------------------------
INVOICE_NUMBER      : ✓
DATE                : ✗
  Ground truth: 02.05.2024
  Prediction:   MISSING
BILL_TO             : ✗
  Ground truth: Rosa Maria Aguado
  Prediction:   MISSING
COMPANY             : ✗
  Ground truth: Aldenaire & Partners
  Prediction:   MISSING
ADDRESS             : ✗
  Ground truth: 123 Anywhere St., Any City, ST 12345
  Prediction:   MISSING
EMAIL               : ✗
  Ground truth: hello@reallygreatsite.com
  Prediction:   MISSING
BANK_NAME           : ✓
BANK_ACCOUNT        : ✗
  Ground truth: 123-456-7890
  Prediction:   MISSING
ITEM_DESCRIPTION    : ✗
  Ground truth: Two days onsite
  Prediction:   MISSING
QTY                 : ✗
  Ground truth: 2
  Prediction:   MISSING
RATE                : ✗
  Ground truth: 640
  Prediction:   MISSING
AMOUNT              : ✗
  Ground truth: 1280.00
  Prediction:   MISSING
SUBTOTAL            : ✗
  Ground truth: $ 2820.00
  Pred

In [38]:
# Show the last interaction for transparency
print("\nInspecting last prompt:")
dspy.inspect_history(n=1)


Inspecting last prompt:




[2025-09-16T13:50:03.809543]

System message:

Your input fields are:
1. `text` (str): Raw invoice text
Your output fields are:
1. `rationale` (str): Brief reasoning, list detected fields
2. `extracted` (dict): JSON dict with only UPPERCASE dataset keys and string values
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## rationale ## ]]
{rationale}

[[ ## extracted ## ]]
{extracted}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "additionalProperties": true}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Extract key-value invoice fields as a JSON dict from free-form invoice text.


User message:

[[ ## text ## ]]
Black and White Minimalist Business Invoice

Invoice16 June 2025

Invoice No. 12345

BILL TO:

Marceline Anderson

+123-456-7890

DESCRIPTION PRICE

TAX (20%)SUBTOTAL

SUBTOTAL

TOTALS

Social Med